<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Gradient Descent in Sklearn

_Authors: Kiefer Katovich (SF)_

---

Until now we've been using specific sklearn model classes to perform regression and classification such as `LinearRegression` and `LogisticRegression`. Unfortunately, while these methods work well on smaller datasets with relatively small numbers of columns, once you start getting into "Medium Data" these slow down to a crawl, and take up so much memory that fitting them becomes mind-numbingly slow (especially on a laptop).

Luckily, sklearn comes with  stochastic gradient descent solvers for regression and classification:
- `SGDRegressor`
- `SGDClassifier`

Due to its ability to minimize the loss function iteratively on smaller portions of the data, it avoids the intense slowdown other models suffer on large datasets.

> **Note:** The gradient descent solvers are very flexible and can fit a variety of different model types not covered here. I highly recommend reading their documentation in detail.

---

### SF assessor data

This lab uses data from the SF assessor's office on housing prices in San Francisco - it's already cleaned up.

You can see that the dataset has 250k rows. When expanding this with dummy-coded categorical columns it can become quite large. Be careful that you don't exceed the memory on your computer.


In [1]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import scipy.stats as stats

import patsy

import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

### 1. Load the data.

Examine the columns.

In [2]:
prop = pd.read_csv('./datasets/assessor_sample.csv')

In [3]:
# A:
prop.nunique()

baths                  4
beds                   5
lot_depth            812
basement_area        749
front_ft              13
owner_pct            562
rooms                  9
property_class         6
neighborhood          88
tax_rate              11
volume                44
sqft                2499
stories                5
year_recorded          8
year_built           110
zone                  71
value             149522
dtype: int64

### 2. Sample down the data

Despite this already being a sample of the full assessor dataset, you should sample the data down further the sake of speed and your computers memory.

Use the `.sample()` function for pandas dataframes to subset this down to < 25000 rows. 

Sampling down large datasets is a common procedure. Finding the optimal parameters with larger subsets of the data may change the hyperparameters and the results, and will get you closer to the best coefficients, but the returns are marginal at a point.

In [4]:
prop_samp = prop.sample(n=25000, random_state = 3)

### 3. Regression with stochastic gradient descent

Below I set up X, y data predicting value (housing price) from the remaining variables. There are ~75,000 rows, with 170 columns.


The `SGDRegressor` is very general and flexible, and can be customized with a variety of keyword arguments.

**Arguments**
- `loss`: `['squared_loss','huber', ...]`
    - The `'squared_loss'` loss corresponds to solving a regression with the least squares loss. This is what I expect you'll use, but there are other options. Huber loss is a "robust" regression loss.
- `penalty`: `['none','l1','l2','elasticnet']`
    - This defines the penalty on the regression that you would like to solve. The l1 and l2 are the Lasso and Ridge, while the elasticnet is the combination of them both.
- `alpha`
    - The regularization strength to be used with a chosen penalty. Same as in Lasso and Ridge.
- `l1_ratio`
    - The mix of the Lasso and Ridge penalties when elasticnet is chosen as the penalty.
- `n_iter`
    - The number of training "epochs" over the data. This is the number of passes that the gradient descent algorithm will make over the data to iteratively fit the weights (defaults to 5).

`SGDRegressor` is most often used in tandem with grid searching to find the optimal parameters for certain models. 

**It is up to you how you want to define the model. You should:**

1. Choose a target to estimate (this should be continuous).
- Select predictors to use.
- Standardize your predictor matrix.
- Build a stochastic gradient descent solver to fit your model. You will likely want to do some kind of gridsearch to find the optimal parameters for your model.
- Describe the model selected through gridsearch and compare the performance to baseline.
- Examine and interpret the coefficients.

In [5]:
prop_samp.drop('owner_pct', inplace = True, axis = 1)
prop_samp.drop('tax_rate', inplace = True, axis = 1)
prop_samp.drop('lot_depth', inplace = True, axis = 1)
prop_samp.drop('basement_area', inplace = True, axis = 1)
prop_samp.drop('front_ft', inplace = True, axis = 1)

In [6]:
prop_samp['age'] = prop_samp.year_recorded - prop_samp.year_built
prop_samp.drop('year_recorded', inplace = True, axis = 1)
prop_samp.drop('year_built', inplace = True, axis = 1)

In [9]:
prop_samp.head()

,baths,beds,rooms,property_class,neighborhood,volume,sqft,stories,zone,value,age
85224,2,4,7,D,02D,16,1883,2,RH1,188245.0,67
73628,1,0,5,D,10H,41,1200,1,RH1,335036.0,83
117363,2,3,5,Z,01E,12,1344,0,RM1,614000.0,30
82519,1,0,5,D,04S,22,1000,1,RH1,92223.0,94
247459,3,0,8,D,02A,15,2012,2,RH1,139555.0,36


In [10]:
# A:
y = prop_samp.value
X = prop_samp[['baths', 'beds', 'rooms', 'volume', 'sqft', 'stories', 'age']]

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [20]:
from sklearn import linear_model
clf = linear_model.SGDRegressor()
clf.fit(X_train, y_train)
print clf.coef_
print clf.intercept_

[ -2.93419584e+10   1.22407302e+11  -3.57739283e+11  -7.31990821e+11
   7.71260916e+11  -1.26036124e+11  -1.12680116e+11]
[ -8.51220615e+10]


In [21]:
clf.score(X_test, y_test)

-9.4179549081516974e+18

In [23]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.16601129389160996

### 4. Classification with stochastic gradient descent

The `SGDClassifier` is very similar to the `SGDRegressor`. The main difference is that the loss functions are changed to regression loss functions.

**Arguments**
- `loss`: `['log', ...]`
    - The `'log'` loss corresponds to solving a logistic regression classifier. This is what I expect you'll use, but there are many other options.
- `penalty`: `['none','l1','l2','elasticnet']`
    - This defines the penalty on the regression that you would like to solve. The l1 and l2 are the Lasso and Ridge, while the elasticnet is the combination of them both.
- `alpha`
    - The regularization strength to be used with a chosen penalty. Same as in Lasso and Ridge.
- `l1_ratio`
    - The mix of the Lasso and Ridge penalties when elasticnet is chosen as the penalty.
- `n_iter`
    - The number of training "epochs" over the data. This is the number of passes that the gradient descent algorithm will make over the data to iteratively fit the weights (defaults to 5).

Like `SGDRegressor`, `SGDClassifier` is most often used in tandem with grid searching to find the optimal parameters for certain models. 

**It is up to you how you want to define the model. You should:**

1. Choose a target to classify (you may need to engineer one from existing variables).
- Calculate the baseline accuracy.
- Select predictors to use.
- Standardize your predictor matrix.
- Build a stochastic gradient descent solver to fit your model. You will likely want to do some kind of gridsearch to find the optimal parameters for your model.
- Describe the model selected through gridsearch and compare the performance to baseline.
- Examine and interpret the coefficients.

In [8]:
# A: